In [1]:
import os

import papermill as pm

In [2]:
# username = "Fro116"
# source = "MAL"
username = "10"
source = "Training"

In [3]:
data_path = "../../data/recommendations"
if not os.path.exists(data_path):
    os.mkdir(data_path)

In [4]:
papermill_path = "../../data/papermill/rec_egress"
if not os.path.exists(papermill_path):
    os.makedirs(papermill_path, exist_ok=True)

In [5]:
def run_ingress_notebook(name):
    pm.execute_notebook(
        f"../RecIngress/{name}.ipynb",
        f"{papermill_path}/{name}.ipynb",
        parameters=dict(username=username, source=source),
    )

## RecIngress

In [6]:
run_ingress_notebook("InitializeRecommendations")

Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

In [7]:
run_ingress_notebook("FetchUserAnimeList")

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

In [8]:
run_ingress_notebook("ImportUserAnimeList")

Executing:   0%|          | 0/14 [00:00<?, ?cell/s]

In [9]:
run_ingress_notebook("ProcessUserAnimeList")

Executing:   0%|          | 0/7 [00:00<?, ?cell/s]

In [10]:
run_ingress_notebook("GenerateSplits")

Executing:   0%|          | 0/9 [00:00<?, ?cell/s]

In [11]:
run_ingress_notebook("CompressUserAnimeList")

Executing:   0%|          | 0/10 [00:00<?, ?cell/s]

Starting kernel event loops.


## RecEgress

In [12]:
# cwd = os.getcwd()
# try:
#     os.chdir("../InferenceAlphas")
#     pm.execute_notebook(
#         "RunAllAlphas.ipynb",
#         f"{data_path}/RunAllAlphas.ipynb",
#         parameters=dict(username=username),
#     )
#     os.chdir("../DisplayRecs")
#     if source == "Training":
#         pm.execute_notebook(
#             "TrainingServingSkew.ipynb",
#             f"{data_path}/TrainingServingSkew.ipynb",
#             parameters=dict(username=username),
#         )
#     else:
#         pm.execute_notebook(
#             "Recommendations.ipynb",
#             f"{data_path}/{username}/Recommendations.ipynb",
#             parameters=dict(username=username),
#         )
# finally:
#     os.chdir(cwd)

In [13]:
print("Success!")

Success!
